### Aikasarjaharjoitustyö datojen poiminta ja yhdistäminen.  
Tässä tiedostossa Käsitellään ilmatieteenlaitoksen data, niin, että sitä voidaan käyttää lineaarisen regression tai muiden algoritmien kanssa.

### Tuodaan datan käsittelyssä ja kuvantamisessa tarvittavat kirjastot. 

In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Säädatan lataus, sarakkeiden poiminta ja datan tarkastelu sekä käsittely

Ladataan tiedot dataframeen. Käytetään materiaalina toisessa yhteydessä hyödynnettyä viinidataa. Muunnetaan ilmatieteenlaitoksen tarjoama aikamuoto formaattiin vvvv-kk-pp tt:mm. Dropataan turhat sarakkeet ja vaihdetaan sarakejärjestys.

In [34]:
df = pd.read_excel('Temp_data.xlsx')

df = df.rename(columns={'Keskituulen nopeus [m/s]':'wind','Lämpötilan keskiarvo [°C]':'C','Aika [Paikallinen aika]':'Aika'}) #Muunnetaan sarakkeiden nimet
df['DS'] = df['Vuosi'].astype(str) + '-' + df['Kuukausi'].astype(str).str.zfill(2) + '-' + df['Päivä'].astype(str).str.zfill(2) + ' ' + df['Aika'] #Tehdään aikamuunnos
df=df.drop(['Havaintoasema','Vuosi','Kuukausi','Päivä','Aika','Ylin lämpötila [°C]','Alin lämpötila [°C]'], axis=1) #Dropataan turhat sarakkeet

df_C = df.loc[:, ['DS', 'C','wind']] # Vaihdetaan sarakejärjestys
print(df_C.head())

                 DS     C wind
0  2019-05-23 00:00  16.4  2.5
1  2019-05-23 01:00  15.6  2.1
2  2019-05-23 02:00  14.8  3.4
3  2019-05-23 03:00  14.8  4.5
4  2019-05-23 04:00  14.8  4.9


Tarkastetaan missä muodossa sarakkeiden data on.

In [35]:
print(df_C.dtypes)


DS      object
C       object
wind    object
dtype: object


Tehdään datatyyppien muunnokset.

In [38]:
df_C['DS'] = pd.to_datetime(df_C['DS'])
df_C['DS'] = df_C['DS'].dt.strftime('%Y-%m-%d %H:%M')
df_C.to_excel('C.xlsx', index=False)
print(df_C)

                     DS     C wind
0      2019-05-23 00:00  16.4  2.5
1      2019-05-23 01:00  15.6  2.1
2      2019-05-23 02:00  14.8  3.4
3      2019-05-23 03:00  14.8  4.5
4      2019-05-23 04:00  14.8  4.9
...                 ...   ...  ...
41099  2024-01-31 19:00   1.4  4.6
41100  2024-01-31 20:00   1.7  6.6
41101  2024-01-31 21:00   1.6  8.9
41102  2024-01-31 22:00   1.2  7.7
41103  2024-01-31 23:00     1  7.2

[41104 rows x 3 columns]


Edellä olevat voidaan myös todentaa tarkastelemalla muutujien tilastollisia tietoja. Jokaisella muuttujalla on variaatiota. (std <> 0 )

In [188]:
df_C.describe()

,C
count,41080.000000
mean,6.848398
std,9.385970
min,-26.000000
25%,0.300000
50%,6.200000
75%,14.100000
max,32.400000


Lämpötiladata näyttää olevan on OK. 